
# AICity Challenge 2025 — Track 4 (FishEye)  
## YOLO11 (Ultralytics) — Train on FishEye8K → Inference on FishEye1Keval → `submission.json`

**What this notebook does**
- Installs latest Ultralytics (YOLO **v11** family).
- Converts **FishEye8K** to YOLO format if it’s in COCO (auto-detect).
- Trains YOLO on FishEye8K.
- Runs inference on **FishEye1Keval (1000 imgs)** and writes `submission.json` in **exact format** required.
- Measures end‑to‑end **FPS** for the 1000 images and reports `FPS` and `NormFPS` (MaxFPS=25).

> **Classes / IDs (fixed by track):** Bus=0, Bike=1, Car=2, Pedestrian=3, Truck=4  
> **Image ID rule (from organizers):** `image_id = int(f"{cameraIndex}{sceneIndex}{frame}")` with scene map `{'M':0,'A':1,'E':2,'N':3}`.

> **Datasets (edit paths below to match your Kaggle Inputs):**  
> `FISHEYE8K_ROOT = /kaggle/input/fisheye8k`  
> `FISHEYE1K_ROOT = /kaggle/input/fisheye1keval`


In [1]:
!pip install -q "ultralytics>=8.3.209"

import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:64"
os.environ["NCCL_P2P_DISABLE"] = "1"
os.environ["NCCL_IB_DISABLE"] = "1"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21

In [2]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [3]:
import torch
print("device_count:", torch.cuda.device_count())
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))


device_count: 0


In [4]:

# ===============================
# 1) ENV & PATHS (EDIT IF NEEDED)
# ===============================
import os, sys, json, time, glob, shutil
from pathlib import Path

# Default Kaggle input paths (rename to match your datasets)
FISHEYE8K_ROOT = Path("/kaggle/input/fisheye8k/Fisheye8K")
FISHEYE1K_ROOT = Path("/kaggle/input/fisheye1k/Fisheye1K")

WORKDIR = Path("/kaggle/working/aicity2025_track4")
WORKDIR.mkdir(parents=True, exist_ok=True)

DATASETS_DIR = WORKDIR / "datasets"
DATASETS_DIR.mkdir(parents=True, exist_ok=True)

SUBMISSION_JSON = WORKDIR / "submission.json"
METRICS_JSON    = WORKDIR / "speed_metrics.json"

CLASS_NAMES = ["Bus","Bike","Car","Pedestrian","Truck"]
CLASS_TO_ID = {n:i for i,n in enumerate(CLASS_NAMES)}

print("FishEye8K root    :", FISHEYE8K_ROOT, "exists:", FISHEYE8K_ROOT.exists())
print("FishEye1Keval root:", FISHEYE1K_ROOT, "exists:", FISHEYE1K_ROOT.exists())
print("Workdir           :", WORKDIR)


FishEye8K root    : /kaggle/input/fisheye8k/Fisheye8K exists: True
FishEye1Keval root: /kaggle/input/fisheye1k/Fisheye1K exists: False
Workdir           : /kaggle/working/aicity2025_track4


In [5]:
import ultralytics, platform
print("Ultralytics:", ultralytics.__version__, "- Python:", platform.python_version())


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics: 8.3.213 - Python: 3.11.13


In [6]:
!nvidia-smi

import torch, os, platform
print("torch:", torch.__version__,
      "| torch.cuda:", torch.version.cuda,
      "| is_available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

/bin/bash: line 1: nvidia-smi: command not found
torch: 2.6.0+cu124 | torch.cuda: 12.4 | is_available: False
CUDA_VISIBLE_DEVICES: 0,1


In [7]:

# ======================================
# 3) UTILS: conversion + image_id helper
# ======================================
from pathlib import Path
from PIL import Image
import numpy as np

SCENE_LIST = ['M','A','E','N']

def get_image_Id(img_name: str) -> int:
    img_name = img_name.split('.png')[0]
    cameraIndx = int(img_name.split('_')[0].replace('camera',''))
    sceneIndx  = SCENE_LIST.index(img_name.split('_')[1])
    frameIndx  = int(img_name.split('_')[2])
    return int(f"{cameraIndx}{sceneIndx}{frameIndx}")

def ensure_dir(p: Path):
    p.mkdir(parents=True, exist_ok=True)

def find_first_existing(paths):
    for p in paths:
        if Path(p).exists(): return Path(p)
    return None

def coco_to_yolo_one_bbox(x, y, w, h, img_w, img_h):
    cx = (x + w/2) / img_w
    cy = (y + h/2) / img_h
    return cx, cy, w/img_w, h/img_h


In [8]:

# =====================================================
# 4) PREPARE FISHEYE8K in YOLO format for Ultralytics
# =====================================================
from tqdm import tqdm
import json

TARGET_YOLO = DATASETS_DIR / "fisheye8k_yolo"
IMG_TRAIN = TARGET_YOLO / "images" / "train"
IMG_VAL   = TARGET_YOLO / "images" / "val"
LBL_TRAIN = TARGET_YOLO / "labels" / "train"
LBL_VAL   = TARGET_YOLO / "labels" / "val"
for p in [IMG_TRAIN, IMG_VAL, LBL_TRAIN, LBL_VAL]: ensure_dir(p)

def copy_all(src_glob, dst_dir: Path):
    for s in glob.glob(str(src_glob)):
        dst = dst_dir / Path(s).name
        if not dst.exists(): shutil.copy2(s, dst)

def convert_coco_split(images_dir: Path, coco_json_path: Path, yolo_img_dir: Path, yolo_lbl_dir: Path):
    print(f"Converting COCO -> YOLO:", coco_json_path.name)
    with open(coco_json_path,'r') as f:
        coco = json.load(f)
    img_dict = {img["id"]: img for img in coco["images"]}
    anns_by_img = {}
    for ann in coco["annotations"]:
        if ann.get("iscrowd",0) == 1: continue
        anns_by_img.setdefault(ann["image_id"], []).append(ann)
    # Category mapping by name (fallback: keep IDs)
    cat_map = {}
    if "categories" in coco:
        for c in coco["categories"]:
            name = c["name"]
            cat_map[c["id"]] = CLASS_TO_ID.get(name, c["id"])
    for img_id, info in tqdm(img_dict.items()):
        fn = info["file_name"]
        src_img = images_dir / fn
        if not src_img.exists():
            # try nested common locations
            cand = find_first_existing([images_dir/"train"/fn, images_dir/"val"/fn, images_dir/"images"/fn])
            if cand is not None: src_img = cand
        if not src_img.exists():
            continue
        dst_img = yolo_img_dir / Path(fn).name
        if not dst_img.exists(): shutil.copy2(src_img, dst_img)
        W = info.get("width"); H = info.get("height")
        if W is None or H is None:
            with Image.open(src_img) as im: W,H = im.size
        lines = []
        for ann in anns_by_img.get(img_id, []):
            x,y,w,h = ann["bbox"]
            cx,cy,nw,nh = coco_to_yolo_one_bbox(x,y,w,h,W,H)
            cat = cat_map.get(ann["category_id"], ann["category_id"])
            lines.append(f"{cat} {cx:.6f} {cy:.6f} {nw:.6f} {nh:.6f}")
        (yolo_lbl_dir / (Path(fn).stem + ".txt")).write_text("\n".join(lines))

# Heuristics: read COCO jsons if present
train_json = FISHEYE8K_ROOT / "train" / "train.json"
val_json   = FISHEYE8K_ROOT / "test"  / "test.json"

images_train_root = FISHEYE8K_ROOT / "train" / "images"
images_val_root   = FISHEYE8K_ROOT / "test"  / "images"

if train_json and val_json:
    convert_coco_split(images_train_root, train_json, IMG_TRAIN, LBL_TRAIN)
    convert_coco_split(images_val_root, val_json,   IMG_VAL,   LBL_VAL)
else:
    # Already YOLO? Copy over standard tree
    for sub in ["train","val"]:
        copy_all(images_root/f"images/{sub}/*", TARGET_YOLO/f"images/{sub}")
        copy_all(images_root/f"labels/{sub}/*", TARGET_YOLO/f"labels/{sub}")

# Write Ultralytics YAML
yaml_text = f"""
# FishEye8K data config for Ultralytics
path: {TARGET_YOLO.as_posix()}
train: images/train
val: images/val
names:
  0: Bus
  1: Bike
  2: Car
  3: Pedestrian
  4: Truck
"""
(TARGET_YOLO / "fisheye8k.yaml").write_text(yaml_text)
print("Data YAML:", (TARGET_YOLO / "fisheye8k.yaml"))
print("Train imgs:", len(list(IMG_TRAIN.glob('*.png'))) + len(list(IMG_TRAIN.glob('*.jpg'))))
print("Val imgs  :", len(list(IMG_VAL.glob('*.png'))) + len(list(IMG_VAL.glob('*.jpg'))))


Converting COCO -> YOLO: train.json


100%|██████████| 5288/5288 [04:19<00:00, 20.41it/s]


Converting COCO -> YOLO: test.json


100%|██████████| 2712/2712 [01:50<00:00, 24.48it/s]


Data YAML: /kaggle/working/aicity2025_track4/datasets/fisheye8k_yolo/fisheye8k.yaml
Train imgs: 5288
Val imgs  : 2712


In [9]:

# ==================================
# 5) TRAIN — Ultralytics YOLO (v11)
# ==================================
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:128"

from ultralytics import YOLO

DATA_YAML = (DATASETS_DIR / "fisheye8k_yolo" / "fisheye8k.yaml").as_posix()

BASE_MODEL = "yolo12l.pt"   # nếu vẫn trục trặc -> "yolo12m.pt"
EPOCHS     = 150
IMG_SIZE   = 704
BATCH      = 12             # 2 GPU => 6/GPU
NBS        = 48             # giảm từ 64 -> 48 để LR hiệu dụng dịu hơn

model = YOLO(BASE_MODEL)
results = model.train(
    data=DATA_YAML,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH,
    nbs=NBS,
    device="0,1",
    workers=2,

    # Ổn định số học & LR
    amp=False,               # ✅ tắt FP16 để tránh NaN sớm
    optimizer="adamw",       # ✅ adamw ổn định hơn SGD đầu train
    lr0=0.001,               # ✅ LR dịu (có thể 0.0008–0.0015)
    lrf=0.01,                # giữ cuối kỳ thấp
    warmup_epochs=5,         # warmup dài hơn

    # Giảm độ “gắt” của augment giai đoạn đầu
    mosaic=0.3,              # 0.5 -> 0.3
    scale=0.35,              # 0.5 -> 0.35
    erasing=0.2,             # 0.4 -> 0.2
    hsv_s=0.5, hsv_v=0.2,    # dịu màu/độ sáng

    # Tránh peak VRAM trong DDP
    compile=False,
    multi_scale=False,

    cos_lr=True,
    patience=25,
    save_period=10,
    cache=False,
)

# Locate best checkpoint
save_dir = Path(results.save_dir) if hasattr(results, "save_dir") else Path("runs/detect/train")
best_pt = save_dir / "weights" / "best.pt"
print("Best checkpoint:", best_pt, "exists:", best_pt.exists())


Ultralytics 8.3.213 🚀 Python-3.11.13 torch-2.6.0+cu124 


ValueError: Invalid CUDA 'device=0,1' requested. Use 'device=cpu' or pass valid CUDA device(s) if available, i.e. 'device=0' or 'device=0,1,2,3' for Multi-GPU.

torch.cuda.is_available(): False
torch.cuda.device_count(): 0
os.environ['CUDA_VISIBLE_DEVICES']: 0,1
See https://pytorch.org/get-started/locally/ for up-to-date torch install instructions if no CUDA devices are seen by torch.


In [ ]:

# =======================
# 6) (Optional) VALIDATE
# =======================
_ = model.val(data=DATA_YAML, imgsz=int(os.environ.get("AICITY_VAL_IMSIZE", "832")),
              device=0 if torch.cuda.is_available() else "cpu")


In [ ]:
# ========================================================
# 7) INFERENCE on FishEye1Keval — create submission.json
# ========================================================
from pathlib import Path
import time, json, torch, gc, os
from ultralytics import YOLO

# Clear VRAM
gc.collect()
torch.cuda.empty_cache()

best_pt = "/kaggle/working/runs/detect/train/weights/best.pt"
FISHEYE1K_ROOT = Path("/kaggle/input/fisheye1k/Fisheye1K")
WORKDIR = Path("/kaggle/working/aicity2025_track4")
SUBMISSION_JSON = WORKDIR / "submission.json"
METRICS_JSON    = WORKDIR / "speed_metrics.json"

assert Path(best_pt).exists(), "best.pt not found"
model = YOLO(best_pt)

IMG_SIZE = 832   # nhỏ hơn để tránh OOM
CONF_THRES = 0.05
IOU_THRES  = 0.5
MAX_DETS   = 300

# Collect test images
test_imgs = sorted([p for ext in ("*.png","*.jpg","*.jpeg") for p in FISHEYE1K_ROOT.rglob(ext)])
print("Test images:", len(test_imgs))
SCENE_LIST = ['M','A','E','N']
def get_image_Id(img_name: str) -> int:
    img_name = img_name.split('.png')[0]
    cameraIndx = int(img_name.split('_')[0].replace('camera',''))
    sceneIndx  = SCENE_LIST.index(img_name.split('_')[1])
    frameIndx  = int(img_name.split('_')[2])
    return int(f"{cameraIndx}{sceneIndx}{frameIndx}")
    
# Inference
start = time.time()
records = []
chunk_size = 50   # nhỏ để tránh OOM
for i in range(0, len(test_imgs), chunk_size):
    subset = test_imgs[i:i+chunk_size]
    pred_gen = model.predict(
        source=[str(p) for p in subset],
        imgsz=IMG_SIZE,
        conf=CONF_THRES,
        iou=IOU_THRES,
        max_det=MAX_DETS,
        device=0,
        verbose=False,
        stream=True,
        batch=2,
        augment=False
    )
    for img_path, res in zip(subset, pred_gen):
        h, w = res.orig_shape[:2]
        image_name = Path(img_path).name
        image_id = get_image_Id(image_name)
        if res.boxes is None or len(res.boxes) == 0:
            continue
        for b in res.boxes:
            x1, y1, x2, y2 = b.xyxy[0].tolist()
            x1 = max(0.0, min(float(x1), w-1))
            y1 = max(0.0, min(float(y1), h-1))
            bw = max(0.0, min(float(x2)-x1, w - x1))
            bh = max(0.0, min(float(y2)-y1, h - y1))
            cls_id = int(b.cls.item())
            score  = float(b.conf.item())
            rec = {
                "image_id": image_id,
                "category_id": cls_id,
                "bbox": [x1, y1, bw, bh],
                "score": score
            }
            records.append(rec)

elapsed = time.time() - start
N = len(test_imgs)
fps = N / elapsed
norm_fps = min(fps, 25.0) / 25.0

with open(SUBMISSION_JSON, "w") as f:
    json.dump(records, f)
with open(METRICS_JSON, "w") as f:
    json.dump({"N": N, "time_sec": elapsed, "FPS": fps, "NormFPS": norm_fps}, f, indent=2)

print(f"Processed {N} images in {elapsed:.2f}s → FPS={fps:.2f}, NormFPS={norm_fps:.3f}")
print("Sample entries:", records[:3])


In [ ]:
# ===============================
# 8) VIZ — Training curves
# ===============================
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, Image as IPyImage

# Tìm thư mục run của lần train hiện tại
train_dir = Path(results.save_dir) if hasattr(results, "save_dir") else Path("runs/detect/train")
csv_path  = train_dir / "results.csv"
png_path  = train_dir / "results.png"

print("Train dir:", train_dir)
if csv_path.exists():
    df = pd.read_csv(csv_path)
    # Vẽ các loss
    for col in [c for c in df.columns if "loss" in c.lower()]:
        plt.figure()
        plt.plot(df["epoch"], df[col])
        plt.title(col)
        plt.xlabel("epoch"); plt.ylabel(col)
        plt.grid(True)
        plt.show()
    # Vẽ các metric (mAP, precision, recall, F1 nếu có)
    metric_cols = [c for c in df.columns if ("map" in c.lower() or "precision" in c.lower() or "recall" in c.lower() or "f1" in c.lower())]
    for col in metric_cols:
        plt.figure()
        plt.plot(df["epoch"], df[col])
        plt.title(col)
        plt.xlabel("epoch"); plt.ylabel(col)
        plt.grid(True)
        plt.show()
else:
    print("Không tìm thấy", csv_path)

# Ultralytics cũng đã render sẵn 1 ảnh tổng hợp
if png_path.exists():
    display(IPyImage(filename=str(png_path)))

In [ ]:
# =========================================
# 9) VIZ — PR / F1 / P / R + Confusion Mat
# =========================================
from ultralytics import YOLO
from IPython.display import display, Image as IPyImage
import glob

# Đảm bảo đang dùng best checkpoint
model = YOLO(str(best_pt)) if Path(best_pt).exists() else model

val_res = model.val(data=DATA_YAML, imgsz=IMG_SIZE, device=0 if torch.cuda.is_available() else "cpu", plots=True)
val_dir = Path(val_res.save_dir)
print("Val dir:", val_dir)

# Hiển thị các hình tiêu chuẩn nếu có
plot_names = [
    "PR_curve.png", "F1_curve.png", "P_curve.png", "R_curve.png",
    "confusion_matrix.png", "confusion_matrix_normalized.png", "results.png"
]
for name in plot_names:
    p = val_dir / name
    if p.exists():
        display(IPyImage(filename=str(p)))


In [ ]:
# ======================================
# 10) VIZ — Sample detections on val set
# ======================================
from pathlib import Path
import random
from IPython.display import display, Image as IPyImage

val_images = sorted([p for ext in ("*.png","*.jpg","*.jpeg") for p in (Path(IMG_VAL)).glob(ext)])
sample_paths = random.sample(val_images, k=min(12, len(val_images)))

viz_dir = WORKDIR / "viz_val_samples"
preds = model.predict(
    source=[str(p) for p in sample_paths],
    imgsz=IMG_SIZE,
    conf=float(os.environ.get("AICITY_CONF", "0.25")),
    iou=float(os.environ.get("AICITY_NMS", "0.7")),
    device=0 if torch.cuda.is_available() else "cpu",
    save=True, project=str(viz_dir), name="pred", exist_ok=True, verbose=False
)

# Hiển thị một vài ảnh kết quả
pred_dir = viz_dir / "pred"
for img_path in sorted(pred_dir.glob("*"))[:12]:
    display(IPyImage(filename=str(img_path)))



### Notes
- The leaderboard will evaluate **F1** with hidden GT. This notebook only reports **FPS** and **NormFPS** as described:  
  \[NormFPS = min(FPS, 25) / 25\].  
- You can tune `AICITY_*` env vars for quick experiments:
  - `AICITY_MODEL` = `yolo11n.pt|yolo11s.pt|yolo11m.pt|yolo11l.pt|yolo11x.pt`
  - `AICITY_EPOCHS` (default 20)
  - `AICITY_BATCH`  (default 16)
  - `AICITY_IMSIZE` (default 960)
  - `AICITY_CONF`   (default 0.25)
  - `AICITY_NMS`    (default 0.7)
  - `AICITY_MAXDETS` (default 300)

**Submission file** to upload: `/kaggle/working/aicity2025_track4/submission.json`
